# Прогнозирование с помощью регрессии

## Загрузка и предобработка данных

In [1]:
import pandas as pd
import numpy as np
import scipy as sc
from scipy import stats
import matplotlib.pyplot as plt
from sklearn import model_selection
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import StandardScaler as scaler
from sklearn.metrics import mean_absolute_error as mae
from itertools import product
import datetime

C:\Program Files\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
C:\Program Files\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [2]:
import xgboost as xgb

In [3]:
import warnings
warnings.simplefilter(action='ignore')

Загружаем список из 102 регионов для прогноза, отобранных на неделе 2

In [3]:
nonzero_reg = pd.read_csv("nonzero_regions.csv").iloc[:,1]

Загружаем аггрегированные данные за январь-июнь 2016. Отбираем для дальнейшего анализа данные только по отобранным 102 регионам

In [4]:
df_1=pd.read_csv("taxi_01_2016.csv")
df_2=pd.read_csv("taxi_02_2016.csv")
df_3=pd.read_csv("taxi_03_2016.csv")
df_4=pd.read_csv("taxi_04_2016.csv")
df_5=pd.read_csv("taxi_05_2016.csv")
df_6=pd.read_csv("taxi_06_2016.csv")
df=pd.concat([ df_1,df_2,df_3,df_4,df_5, df_6], axis=0)
df

,Unnamed: 0,1,2,3,4,5,6,7,8,9,...,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500
0,2016-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2016-01-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2016-01-01 02:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2016-01-01 03:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2016-01-01 04:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,2016-06-30 19:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
716,2016-06-30 20:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
717,2016-06-30 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
718,2016-06-30 22:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
df.rename(columns={'Unnamed: 0':'time'}, inplace=True)
df.index=df.time
df=df.iloc[:,nonzero_reg]
df.head()

,1075,1076,1077,1125,1126,1127,1128,1129,1130,1131,...,1630,1684,1733,1734,1783,2068,2069,2118,2119,2168
time,,,,,,,,,,,,,,,,,,,,,
2016-01-01 00:00:00,80.0,144.0,50.0,77.0,319.0,402.0,531.0,617.0,846.0,267.0,...,12.0,0.0,2.0,44.0,5.0,41.0,4.0,70.0,7.0,66.0
2016-01-01 01:00:00,91.0,211.0,49.0,134.0,404.0,420.0,370.0,453.0,594.0,224.0,...,29.0,0.0,5.0,2.0,2.0,4.0,0.0,47.0,1.0,29.0
2016-01-01 02:00:00,90.0,146.0,23.0,110.0,393.0,425.0,313.0,366.0,377.0,138.0,...,47.0,0.0,3.0,0.0,4.0,0.0,0.0,69.0,1.0,14.0
2016-01-01 03:00:00,32.0,87.0,16.0,62.0,252.0,399.0,324.0,309.0,327.0,166.0,...,46.0,0.0,2.0,4.0,5.0,1.0,0.0,21.0,0.0,9.0
2016-01-01 04:00:00,24.0,43.0,10.0,53.0,145.0,254.0,264.0,333.0,318.0,145.0,...,43.0,0.0,0.0,1.0,1.0,0.0,0.0,26.0,1.0,6.0


Загружаем датасет с прогнозами ARIMA на январь-июнь, полученным на прошлой неделе

In [6]:
forecast=pd.read_csv('forecast.csv')
forecast.head()
forecast.index=forecast.time
forecast=forecast.drop(['time'], axis=1)
forecast.head()

,forecast_1,forecast_2,forecast_3,forecast_4,forecast_5,forecast_6,region
time,,,,,,,
2016-01-01 00:00:00,4.376683,4.826494,3.196990,2.524114,4.825234,5.174734,1435
2016-01-01 01:00:00,19.951172,17.838943,16.698748,18.547465,18.459001,24.245501,1435
2016-01-01 02:00:00,41.120005,39.236746,40.366111,39.581274,44.693627,47.736560,1435
2016-01-01 03:00:00,27.503669,29.007512,28.585211,34.048519,37.431204,36.497186,1435
2016-01-01 04:00:00,20.775319,20.615761,26.333432,29.962355,29.266716,22.562023,1435


## Формирование тренировочного, валидационного и тестового датасетов 

In [7]:
time = pd.to_datetime(df.index)
nt = np.arange(1, df.shape[0] + 1)

ndf = {}

for elem in list(nonzero_reg):
    ndf[elem] = pd.DataFrame({'yt': df[str(elem)]})

    ndf[elem]['reg'] = str(elem)

    ndf[elem]['month'] = time.month
    ndf[elem]['day'] = time.day
    ndf[elem]['weekday'] = time.weekday
    ndf[elem]['hour'] = time.hour

    # Число поездок в предыдущие 10 часов
    for i in range(1, 11):
        ndf[elem]['yt' + str(i)] = ndf[elem]['yt'].shift(i)

    for i in range(1, 3):
        ndf[elem]['y_day' + str(i)] = ndf[elem]['yt'].shift(24 * i)

    # Элементы ряда Фурье
    for i in range(1, 30):
        ndf[elem]['s' + str(i)] = np.sin(2 * i * np.pi * nt / 168)
        ndf[elem]['c' + str(i)] = np.cos(2 * i * np.pi * nt / 168)

    # Истинные значения
    for i in range(0, 6):
        ndf[elem]['y' + str(i + 1)] = ndf[elem]['yt'].shift(-i - 1)

    ndf[elem] = pd.concat([ndf[elem], forecast[forecast['region'] == elem]], axis = 1)
    ndf[elem] = ndf[elem].drop(['region'], axis = 1)

  

In [8]:
ndf[nonzero_reg[0]]

,yt,reg,month,day,weekday,hour,yt1,yt2,yt3,yt4,...,y3,y4,y5,y6,forecast_1,forecast_2,forecast_3,forecast_4,forecast_5,forecast_6
time,,,,,,,,,,,,,,,,,,,,,
2016-01-01 00:00:00,80.0,1075,1,1,4,0,NaN,NaN,NaN,NaN,...,32.0,24.0,11.0,7.0,50.357521,27.100380,15.408897,11.012560,4.850987,11.426459
2016-01-01 01:00:00,91.0,1075,1,1,4,1,80.0,NaN,NaN,NaN,...,24.0,11.0,7.0,9.0,65.821117,56.897958,53.333806,47.410342,54.041647,65.757691
2016-01-01 02:00:00,90.0,1075,1,1,4,2,91.0,80.0,NaN,NaN,...,11.0,7.0,9.0,18.0,89.785074,88.571796,83.354808,90.188100,101.951243,123.416544
2016-01-01 03:00:00,32.0,1075,1,1,4,3,90.0,91.0,80.0,NaN,...,7.0,9.0,18.0,22.0,88.852517,83.655597,90.494920,102.259789,123.725493,150.081465
2016-01-01 04:00:00,24.0,1075,1,1,4,4,32.0,90.0,91.0,80.0,...,9.0,18.0,22.0,27.0,9.398996,10.929996,21.099398,42.108460,68.358024,68.318518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-06-30 19:00:00,116.0,1075,6,30,3,19,88.0,82.0,81.0,86.0,...,106.0,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-06-30 20:00:00,104.0,1075,6,30,3,20,116.0,88.0,82.0,81.0,...,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-06-30 21:00:00,151.0,1075,6,30,3,21,104.0,116.0,88.0,82.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Разделим датасеты на train, valid, test

In [9]:
dataTrain = {}
dataValid = {}
dataTest = {}
for elem in nonzero_reg:
    dataTrain[elem] = ndf[elem].iloc[:2904]
    dataValid[elem] = ndf[elem].iloc[2904 : 3648]
    dataTest[elem] = ndf[elem].iloc[3648:]

In [10]:
xTrain = pd.DataFrame()
xValid = pd.DataFrame()
xTest = pd.DataFrame()
for elem in list(nonzero_reg):
    xTrain = pd.concat([xTrain, dataTrain[elem]], axis = 0)
    xValid = pd.concat([xValid, dataValid[elem]], axis = 0)
    xTest = pd.concat([xTest, dataTest[elem]], axis = 0)

In [11]:
xTrain.head()

,yt,reg,month,day,weekday,hour,yt1,yt2,yt3,yt4,...,y3,y4,y5,y6,forecast_1,forecast_2,forecast_3,forecast_4,forecast_5,forecast_6
time,,,,,,,,,,,,,,,,,,,,,
2016-01-01 00:00:00,80.0,1075,1,1,4,0,NaN,NaN,NaN,NaN,...,32.0,24.0,11.0,7.0,50.357521,27.100380,15.408897,11.012560,4.850987,11.426459
2016-01-01 01:00:00,91.0,1075,1,1,4,1,80.0,NaN,NaN,NaN,...,24.0,11.0,7.0,9.0,65.821117,56.897958,53.333806,47.410342,54.041647,65.757691
2016-01-01 02:00:00,90.0,1075,1,1,4,2,91.0,80.0,NaN,NaN,...,11.0,7.0,9.0,18.0,89.785074,88.571796,83.354808,90.188100,101.951243,123.416544
2016-01-01 03:00:00,32.0,1075,1,1,4,3,90.0,91.0,80.0,NaN,...,7.0,9.0,18.0,22.0,88.852517,83.655597,90.494920,102.259789,123.725493,150.081465
2016-01-01 04:00:00,24.0,1075,1,1,4,4,32.0,90.0,91.0,80.0,...,9.0,18.0,22.0,27.0,9.398996,10.929996,21.099398,42.108460,68.358024,68.318518


Используем регион в качестве категориального признака, для этого закодируем его

In [12]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [15]:
encoder = OneHotEncoder(drop = 'first')
regions = []
for elem in nonzero_reg[:-1]:
    regions.append('r_' + str(elem)) 

regEncTrain = pd.DataFrame(encoder.fit_transform(xTrain[['reg']]).toarray(), columns = regions, index = xTrain.index)
regEncValid = pd.DataFrame(encoder.fit_transform(xValid[['reg']]).toarray(), columns = regions, index = xValid.index)
regEncTest = pd.DataFrame(encoder.fit_transform(xTest[['reg']]).toarray(), columns = regions, index = xTest.index)

xTrain = pd.concat([xTrain.drop(['reg'], axis = 1), regEncTrain], axis = 1)
xValid = pd.concat([xValid.drop(['reg'], axis = 1), regEncValid], axis = 1)
xTest = pd.concat([xTest.drop(['reg'], axis = 1), regEncTest], axis = 1)

In [16]:
xTrain

,yt,month,day,weekday,hour,yt1,yt2,yt3,yt4,yt5,...,r_1580,r_1630,r_1684,r_1733,r_1734,r_1783,r_2068,r_2069,r_2118,r_2119
time,,,,,,,,,,,,,,,,,,,,,
2016-01-01 00:00:00,80.0,1,1,4,0,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-01 01:00:00,91.0,1,1,4,1,80.0,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-01 02:00:00,90.0,1,1,4,2,91.0,80.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-01 03:00:00,32.0,1,1,4,3,90.0,91.0,80.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-01 04:00:00,24.0,1,1,4,4,32.0,90.0,91.0,80.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-04-30 19:00:00,84.0,4,30,5,19,78.0,81.0,77.0,41.0,64.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2016-04-30 20:00:00,120.0,4,30,5,20,84.0,78.0,81.0,77.0,41.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2016-04-30 21:00:00,117.0,4,30,5,21,120.0,84.0,78.0,81.0,77.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


## Обучение прогнозных моделей и проверка их качества на валидационном датасете

In [17]:
respCol = {'y1', 'y2', 'y3', 'y4', 'y5', 'y6'}

In [18]:
respTrain = xTrain.dropna()[respCol]
train = xTrain.dropna().drop(respCol, axis = 1)

respValid = xValid.dropna()[respCol]
valid = xValid.dropna().drop(respCol, axis = 1)

respTest = xTest.dropna()[respCol]
test = xTest.dropna().drop(respCol, axis = 1)

C:\Users\poklo\AppData\Local\Temp/ipykernel_5608/620177850.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  respTrain = xTrain.dropna()[respCol]
C:\Users\poklo\AppData\Local\Temp/ipykernel_5608/620177850.py:4: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  respValid = xValid.dropna()[respCol]
C:\Users\poklo\AppData\Local\Temp/ipykernel_5608/620177850.py:7: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  respTest = xTest.dropna()[respCol]


In [19]:
xReg = {}
for i in range(6):
    xReg[i] = xgb.XGBRegressor()

Обучение модели

In [20]:
model = {}

for (i, y) in zip(range(6), list(respCol)):
    model[i] = xReg[i].fit(train, respTrain[y])

Получение прогнозов

In [21]:
predictVal = pd.DataFrame()
for i in range(6):
    predictVal[i] = model[i].predict(valid)

In [22]:
predictVal

,0,1,2,3,4,5
0,29.676634,47.778664,25.336273,15.443535,16.394176,20.776518
1,18.220503,25.341316,27.833195,16.336863,11.030165,14.058267
2,11.974065,14.278600,29.183105,19.776014,13.208576,8.962940
3,5.822258,6.331695,47.316448,27.437071,15.965167,10.847147
4,12.867502,5.870425,67.575150,42.457085,26.231573,16.601641
...,...,...,...,...,...,...
75883,138.668716,112.372696,60.719769,99.244011,93.624725,125.645256
75884,129.765518,133.419678,25.460865,53.764603,95.145592,91.047600
75885,93.611198,127.385078,6.569302,15.150028,50.038078,92.255173
75886,92.888580,97.363304,16.512964,16.803217,14.664470,54.361740


Оценим качество прогнозов

In [23]:
for (i, y) in zip(range(6), list(respCol)):
    print(str(i) + ' MAE: ', mae(predictVal[i], respValid[y]))

0 MAE:  16.556953202524344
1 MAE:  14.420846631865555
2 MAE:  21.179129348223423
3 MAE:  18.35271615812708
4 MAE:  18.187960522884214
5 MAE:  17.641322763243263


In [24]:
Qmay = 0
for (i, y) in zip(range(6), list(respCol)):
    Qmay += np.abs(respValid[y].values - predictVal[i].values)[:-5].sum()
print('Qmay: ', Qmay / (739 * 6 * 102))

Qmay:  17.842145550763874


Ошибка полученных прогнозов за май по сравнению с ошибкой модели ARIMA, уменьшилась. 17.842145550763874 по сравнению с 20.0837735749116231

## Подбор оптимальных значений гиперпараметров

In [25]:
import hyperopt.hp as hp
from hyperopt import fmin, tpe, STATUS_OK, Trials

Словарь содержит в качестве ключей названия гиперпараметров регрессионных моделей, а в качестве значений - диапазон изменения гиперпараметров.

In [26]:
space={'max_depth': hp.quniform("max_depth", 3, 10, 2),
        'reg_alpha' : hp.quniform('reg_alpha', 50,150,25),
         'n_estimators': hp.quniform("n_estimators", 10, 110, 50),
        'seed': 0
    }

### Прогнозирование на 1 час вперёд

Функция objective() обучает модель с фиксированными значениями гиперпараметров на обучающей выборке и находит значение выбранной метрики качества на тестовой выборке.

In [27]:
def objective(space):
    
    regression = xgb.XGBRegressor(
                    n_estimators = int(space['n_estimators']), max_depth = int(space['max_depth']),reg_alpha = int(space['reg_alpha'])
                    )
    evaluation = [(train, respTrain['y1']), (valid, respValid['y1'])]
    
    regression.fit(train, respTrain['y1'], eval_set = evaluation, eval_metric="mae", early_stopping_rounds = 10, verbose = 0)

    pred = regression.predict(valid)
    mae_pred = mae(respValid['y1'], pred)
    print ("SCORE:", mae_pred)
    return {'loss': mae_pred, 'status': STATUS_OK}

In [28]:
trials = Trials()

Поиск оптимальных значений гиперпараметров с помощью функции `fmin`

In [150]:
%%time
best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)

SCORE:                                                                                                                 
14.41352104713982                                                                                                      
SCORE:                                                                                                                 
14.557174659753445                                                                                                     
SCORE:                                                                                                                 
150.51525932953825                                                                                                     
SCORE:                                                                                                                 
14.37765071477402                                                                                                      
SCORE:                                  

SCORE:                                                                                                                 
14.351751923419467                                                                                                     
SCORE:                                                                                                                 
150.51525932953825                                                                                                     
SCORE:                                                                                                                 
14.351751923419467                                                                                                     
SCORE:                                                                                                                 
150.51525932953825                                                                                                     
SCORE:                                  

In [151]:
print(best_hyperparams)

{'max_depth': 10.0, 'n_estimators': 100.0, 'reg_alpha': 125.0}


Обученик регрессии с оптимальными гиперпаметрами и оцека её качества на валидационной и тестовой выборках

In [153]:
xgReg = xgb.XGBRegressor(max_depth = int(best_hyperparams['max_depth']), n_estimators = int(best_hyperparams['n_estimators']), reg_alpha = int(best_hyperparams['reg_alpha']))
mod0 = xgReg.fit(train, respTrain['y1'])
predictValid0 = xgReg.predict(valid)

In [147]:
predictTest0 = xgReg.predict(test)

In [157]:
QJune = np.abs(respTest['y1'].values - predictTest0).sum()
print('Значение ошибки QJune на тестовой выборке: ', QJune / (714 * 102))

Значение ошибки QJune на тестовой выборке:  14.777816260222885


In [159]:
QJune = np.abs(respValid['y1'].values - predictValid0).sum()
print('Значение ошибки QJune на валидационной выборке: ', QJune / (714 * 102))

Значение ошибки QJune на валидационной выборке:  15.105706563051873


### Прогнозирование на 2 час вперёд

In [29]:
def objective(space):
    
    regression = xgb.XGBRegressor(
                    n_estimators = int(space['n_estimators']), max_depth = int(space['max_depth']),reg_alpha = int(space['reg_alpha'])
                    )
    evaluation = [(train, respTrain['y2']), (valid, respValid['y2'])]
    
    regression.fit(train, respTrain['y2'], eval_set = evaluation, eval_metric="mae", early_stopping_rounds = 10, verbose = 0)

    pred = regression.predict(valid)
    mae_pred = mae(respValid['y2'], pred)
    print ("SCORE:", mae_pred)
    return {'loss': mae_pred, 'status': STATUS_OK}

In [30]:
%%time
best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)

  0%|                                                                          | 0/100 [00:00<?, ?trial/s, best loss=?]

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                                                                                 
16.38493932917174                                                                                                      
  1%|▍                                             | 1/100 [02:15<3:43:37, 135.53s/trial, best loss: 16.38493932917174]

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                                                                                 
150.30968005481762                                                                                                     
  2%|▉                                              | 2/100 [02:18<1:34:23, 57.79s/trial, best loss: 16.38493932917174]

...

SCORE:                                                                                                                 
150.30968005481762                                                                                                     
 98%|███████████████████████████████████████████████ | 98/100 [32:28<00:05,  2.94s/trial, best loss: 16.36618320858471]

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                                                                                 
150.30968005481762                                                                                                     
 99%|███████████████████████████████████████████████▌| 99/100 [32:31<00:02,  2.93s/trial, best loss: 16.36618320858471]

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                                                                                 
150.30968005481762                                                                                                     
100%|███████████████████████████████████████████████| 100/100 [32:34<00:00, 19.55s/trial, best loss: 16.36618320858471]
Wall time: 32min 34s


In [31]:
print(best_hyperparams)

{'max_depth': 10.0, 'n_estimators': 50.0, 'reg_alpha': 125.0}


In [32]:
xgReg = xgb.XGBRegressor(max_depth = int(best_hyperparams['max_depth']), n_estimators = int(best_hyperparams['n_estimators']), reg_alpha = int(best_hyperparams['reg_alpha']))
mod1 = xgReg.fit(train, respTrain['y2'])
predictValid1 = xgReg.predict(valid)

In [33]:
predictTest1 = xgReg.predict(test)

In [34]:
QJune = np.abs(respTest['y2'].values - predictTest1).sum()
print('Значение ошибки QJune на тестовой выборке: ', QJune / (714 * 102))

QJune = np.abs(respValid['y2'].values - predictValid1).sum()
print('Значение ошибки QJune на валидационной выборке: ', QJune / (714 * 102))

Значение ошибки QJune на тестовой выборке:  16.615039973930788
Значение ошибки QJune на валидационной выборке:  17.24368928973524


### Прогнозирование на 3 час вперёд

In [65]:
trials = Trials()

In [66]:
def objective(space):
    
    regression = xgb.XGBRegressor(
                    n_estimators = int(space['n_estimators']), max_depth = int(space['max_depth']),reg_alpha = int(space['reg_alpha'])
                    )
    evaluation = [(train, respTrain['y3']), (valid, respValid['y3'])]
    
    regression.fit(train, respTrain['y3'], eval_set = evaluation, eval_metric="mae", early_stopping_rounds = 10, verbose = 0)

    pred = regression.predict(valid)
    mae_pred = mae(respValid['y3'], pred)
    print ("SCORE:", mae_pred)
    return {'loss': mae_pred, 'status': STATUS_OK}

In [67]:
%%time
best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)

  0%|                                                                          | 0/100 [00:00<?, ?trial/s, best loss=?]

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                                                                                 
17.80774677498963                                                                                                      
  1%|▍                                                | 1/100 [00:30<50:45, 30.76s/trial, best loss: 17.80774677498963]

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                                                                                 
17.444388254891315                                                                                                     
  2%|▉                                             | 2/100 [01:58<1:45:23, 64.52s/trial, best loss: 17.444388254891315]

...

SCORE:                                                                                                                 
150.12223276407337                                                                                                     
 98%|██████████████████████████████████████████████ | 98/100 [26:49<00:22, 11.13s/trial, best loss: 17.433514843939427]

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                                                                                 
150.12223276407337                                                                                                     
 99%|██████████████████████████████████████████████▌| 99/100 [26:52<00:08,  8.72s/trial, best loss: 17.433514843939427]

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                                                                                 
150.12223276407337                                                                                                     
100%|██████████████████████████████████████████████| 100/100 [26:55<00:00, 16.15s/trial, best loss: 17.433514843939427]
Wall time: 26min 55s


In [68]:
print(best_hyperparams)

{'max_depth': 8.0, 'n_estimators': 50.0, 'reg_alpha': 150.0}


In [69]:
xgReg = xgb.XGBRegressor(max_depth = int(best_hyperparams['max_depth']), n_estimators = int(best_hyperparams['n_estimators']), reg_alpha = int(best_hyperparams['reg_alpha']))
mod2 = xgReg.fit(train, respTrain['y3'])
predictValid2 = xgReg.predict(valid)

In [70]:
predictTest2 = xgReg.predict(test)

In [71]:
QJune = np.abs(respTest['y3'].values - predictTest2).sum()
print('Значение ошибки QJune на тестовой выборке: ', QJune / (714 * 102))

QJune = np.abs(respValid['y3'].values - predictValid2).sum()
print('Значение ошибки QJune на валидационной выборке: ', QJune / (714 * 102))

Значение ошибки QJune на тестовой выборке:  17.55115174207085
Значение ошибки QJune на валидационной выборке:  18.283873926067926


### Прогнозирование на 4 час вперёд

In [72]:
trials = Trials()

In [73]:
def objective(space):
    
    regression = xgb.XGBRegressor(
                    n_estimators = int(space['n_estimators']), max_depth = int(space['max_depth']),reg_alpha = int(space['reg_alpha'])
                    )
    evaluation = [(train, respTrain['y4']), (valid, respValid['y4'])]
    
    regression.fit(train, respTrain['y4'], eval_set = evaluation, eval_metric="mae", early_stopping_rounds = 10, verbose = 0)

    pred = regression.predict(valid)
    mae_pred = mae(respValid['y4'], pred)
    print ("SCORE:", mae_pred)
    return {'loss': mae_pred, 'status': STATUS_OK}

In [74]:
%%time
best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)

  0%|                                                                          | 0/100 [00:00<?, ?trial/s, best loss=?]

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                                                                                 
18.08448800490472                                                                                                      
  1%|▍                                                | 1/100 [00:28<46:18, 28.07s/trial, best loss: 18.08448800490472]

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                                                                                 
17.860765382432305                                                                                                     
  2%|▉                                             | 2/100 [01:36<1:24:38, 51.83s/trial, best loss: 17.860765382432305]

...

SCORE:                                                                                                                 
149.98047122074635                                                                                                     
 98%|███████████████████████████████████████████████ | 98/100 [22:50<00:05,  2.91s/trial, best loss: 17.80842153521087]

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                                                                                 
149.98047122074635                                                                                                     
 99%|███████████████████████████████████████████████▌| 99/100 [22:53<00:02,  2.82s/trial, best loss: 17.80842153521087]

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                                                                                 
149.98047122074635                                                                                                     
100%|███████████████████████████████████████████████| 100/100 [22:56<00:00, 13.76s/trial, best loss: 17.80842153521087]
Wall time: 22min 56s


In [75]:
print(best_hyperparams)

{'max_depth': 8.0, 'n_estimators': 50.0, 'reg_alpha': 75.0}


In [76]:
xgReg = xgb.XGBRegressor(max_depth = int(best_hyperparams['max_depth']), n_estimators = int(best_hyperparams['n_estimators']), reg_alpha = int(best_hyperparams['reg_alpha']))
mod3 = xgReg.fit(train, respTrain['y4'])
predictValid3 = xgReg.predict(valid)

In [77]:
predictTest3 = xgReg.predict(test)

In [78]:
QJune = np.abs(respTest['y4'].values - predictTest3).sum()
print('Значение ошибки QJune на тестовой выборке: ', QJune / (714 * 102))

QJune = np.abs(respValid['y4'].values - predictValid3).sum()
print('Значение ошибки QJune на валидационной выборке: ', QJune / (714 * 102))

Значение ошибки QJune на тестовой выборке:  18.00571023976421
Значение ошибки QJune на валидационной выборке:  18.80168762163505


### Прогнозирование на 5 час вперёд

In [79]:
trials = Trials()

In [80]:
def objective(space):
    
    regression = xgb.XGBRegressor(
                    n_estimators = int(space['n_estimators']), max_depth = int(space['max_depth']),reg_alpha = int(space['reg_alpha'])
                    )
    evaluation = [(train, respTrain['y5']), (valid, respValid['y5'])]
    
    regression.fit(train, respTrain['y5'], eval_set = evaluation, eval_metric="mae", early_stopping_rounds = 10, verbose = 0)

    pred = regression.predict(valid)
    mae_pred = mae(respValid['y5'], pred)
    print ("SCORE:", mae_pred)
    return {'loss': mae_pred, 'status': STATUS_OK}

In [81]:
%%time
best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)

  0%|                                                                          | 0/100 [00:00<?, ?trial/s, best loss=?]

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                                                                                 
17.980954019776924                                                                                                     
  1%|▍                                             | 1/100 [00:55<1:32:14, 55.90s/trial, best loss: 17.980954019776924]

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                                                                                 
18.14004145076625                                                                                                      
  2%|▉                                             | 2/100 [01:25<1:06:27, 40.69s/trial, best loss: 17.980954019776924]

...

SCORE:                                                                                                                 
18.143759262893564                                                                                                     
 98%|██████████████████████████████████████████████ | 98/100 [25:53<00:35, 17.62s/trial, best loss: 17.909026830744423]

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                                                                                 
149.9040559772296                                                                                                      
 99%|██████████████████████████████████████████████▌| 99/100 [25:56<00:13, 13.21s/trial, best loss: 17.909026830744423]

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                                                                                 
17.995276872563466                                                                                                     
100%|██████████████████████████████████████████████| 100/100 [26:50<00:00, 16.10s/trial, best loss: 17.909026830744423]
Wall time: 26min 50s


In [82]:
print(best_hyperparams)

{'max_depth': 6.0, 'n_estimators': 50.0, 'reg_alpha': 125.0}


In [83]:
xgReg = xgb.XGBRegressor(max_depth = int(best_hyperparams['max_depth']), n_estimators = int(best_hyperparams['n_estimators']), reg_alpha = int(best_hyperparams['reg_alpha']))
mod4 = xgReg.fit(train, respTrain['y5'])
predictValid4 = xgReg.predict(valid)

In [84]:
predictTest4 = xgReg.predict(test)

In [85]:
QJune = np.abs(respTest['y5'].values - predictTest4).sum()
print('Значение ошибки QJune на тестовой выборке: ', QJune / (714 * 102))

QJune = np.abs(respValid['y5'].values - predictValid4).sum()
print('Значение ошибки QJune на валидационной выборке: ', QJune / (714 * 102))

Значение ошибки QJune на тестовой выборке:  18.388497486494074
Значение ошибки QJune на валидационной выборке:  18.989602077929234


### Прогнозирование на 6 час вперёд

In [86]:
trials = Trials()

In [87]:
def objective(space):
    
    regression = xgb.XGBRegressor(
                    n_estimators = int(space['n_estimators']), max_depth = int(space['max_depth']),reg_alpha = int(space['reg_alpha'])
                    )
    evaluation = [(train, respTrain['y6']), (valid, respValid['y6'])]
    
    regression.fit(train, respTrain['y6'], eval_set = evaluation, eval_metric="mae", early_stopping_rounds = 10, verbose = 0)

    pred = regression.predict(valid)
    mae_pred = mae(respValid['y6'], pred)
    print ("SCORE:", mae_pred)
    return {'loss': mae_pred, 'status': STATUS_OK}

In [88]:
%%time
best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)

  0%|                                                                          | 0/100 [00:00<?, ?trial/s, best loss=?]

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                                                                                 
22.787174387883173                                                                                                     
  1%|▍                                            | 1/100 [02:06<3:29:23, 126.90s/trial, best loss: 22.787174387883173]

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                                                                                 
21.762542688141362                                                                                                     
  2%|▉                                            | 2/100 [03:56<3:10:21, 116.54s/trial, best loss: 21.762542688141362]

...

SCORE:                                                                                                                 
149.90901064726967                                                                                                     
 98%|██████████████████████████████████████████████ | 98/100 [54:02<00:05,  2.71s/trial, best loss: 20.238610441511813]

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                                                                                 
149.90901064726967                                                                                                     
 99%|██████████████████████████████████████████████▌| 99/100 [54:05<00:02,  2.81s/trial, best loss: 20.238610441511813]

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\poklo\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                                                                                 
149.90901064726967                                                                                                     
100%|██████████████████████████████████████████████| 100/100 [54:08<00:00, 32.49s/trial, best loss: 20.238610441511813]
Wall time: 54min 8s


In [89]:
print(best_hyperparams)

{'max_depth': 10.0, 'n_estimators': 50.0, 'reg_alpha': 75.0}


In [90]:
xgReg = xgb.XGBRegressor(max_depth = int(best_hyperparams['max_depth']), n_estimators = int(best_hyperparams['n_estimators']), reg_alpha = int(best_hyperparams['reg_alpha']))
mod5 = xgReg.fit(train, respTrain['y6'])
predictValid5 = xgReg.predict(valid)

In [91]:
predictTest5 = xgReg.predict(test)

In [92]:
QJune = np.abs(respTest['y6'].values - predictTest5).sum()
print('Значение ошибки QJune на тестовой выборке: ', QJune / (714 * 102))

QJune = np.abs(respValid['y6'].values - predictValid5).sum()
print('Значение ошибки QJune на валидационной выборке: ', QJune / (714 * 102))

Значение ошибки QJune на тестовой выборке:  20.431221719050548
Значение ошибки QJune на валидационной выборке:  21.08897222476861


Сохранение прогнозов в файл

In [196]:
forecast = pd.DataFrame(columns=['id', 'y'])

In [ ]:
for i in range(102):
    for j in range(714):
        forecast = forecast.append( {'id': str(nonzero_reg[i]) + '_' + pd.to_datetime(respTest[:714].index[j]).strftime('%Y-%m-%d_%#H') + '_1', 'y' : predictTest0[i * 714 + j]}, ignore_index=True)
        
        forecast = forecast.append( {'id': str(nonzero_reg[i]) + '_' + pd.to_datetime(respTest[:714].index[j]).strftime('%Y-%m-%d_%#H') + '_2', 'y' : predictTest1[i * 714 + j]}, ignore_index=True)
        
        forecast = forecast.append( {'id': str(nonzero_reg[i]) + '_' + pd.to_datetime(respTest[:714].index[j]).strftime('%Y-%m-%d_%#H') + '_3', 'y' : predictTest2[i * 714 + j]}, ignore_index=True)
        
        forecast = forecast.append( {'id': str(nonzero_reg[i]) + '_' + pd.to_datetime(respTest[:714].index[j]).strftime('%Y-%m-%d_%#H') + '_4', 'y' : predictTest3[i * 714 + j]}, ignore_index=True)
        
        forecast = forecast.append( {'id': str(nonzero_reg[i]) + '_' + pd.to_datetime(respTest[:714].index[j]).strftime('%Y-%m-%d_%#H') + '_5', 'y' : predictTest4[i * 714 + j]}, ignore_index=True)
        
        forecast = forecast.append( {'id': str(nonzero_reg[i]) + '_' + pd.to_datetime(respTest[:714].index[j]).strftime('%Y-%m-%d_%#H') + '_6', 'y' : predictTest5[i * 714 + j]}, ignore_index=True)

In [ ]:
forecast.to_csv('forecast.csv', index=False)